<div style="text-align: center;">
    <h1>Implementing Data Encryption Standard w/ Python!</h1>
</div>

In [1]:
# Initial permutation table
ip_table = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# PC1 permutation table
pc1_table = [
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

# Define the left shift schedule for each round
shift_schedule = [1, 1, 2, 2,
                  2, 2, 2, 2,
                  1, 2, 2, 2,
                  2, 2, 2, 1]

# PC2 permutation table
pc2_table = [
    14, 17, 11, 24, 1, 5, 3, 28,
    15, 6, 21, 10, 23, 19, 12, 4,
    26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
]

# Expansion table
e_box_table = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

# S-box tables for DES
s_boxes = [
    # S-box 1
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    # S-box 2
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    # S-box 3
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    # S-box 4
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    # S-box 5
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    # S-box 6
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    # S-box 7
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 9, 2, 8, 0, 5, 6],
        [6, 11, 13, 2, 4, 14, 3, 8, 10, 15, 1, 7, 9, 5, 0, 12]
    ],
    # S-box 8
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 6, 3, 5, 0, 11]
    ]
]

# P permutation table
p_table = [
    16, 7, 20, 21,
    29, 12, 28, 17,
    1, 15, 23, 26,
    5, 18, 31, 10,
    2, 8, 24, 14,
    32, 27, 3, 9,
    19, 13, 30, 6,
    22, 11, 4, 25
]

# Inverse initial permutation table
ip_inv_table = [
    40, 8, 48, 16, 56, 24, 32, 0,
    41, 9, 49, 17, 57, 25, 33, 1,
    42, 10, 50, 18, 58, 26, 34, 2,
    43, 11, 51, 19, 59, 27, 35, 3,
    44, 12, 52, 20, 60, 28, 36, 4,
    45, 13, 53, 21, 61, 29, 37, 5,
    46, 14, 54, 22, 62, 30, 38, 6,
    47, 15, 55, 23, 63, 31, 39, 7
]

In [2]:
def permute(bits, table):
    """ Permute bits according to the given table. """
    return [bits[x - 1] for x in table]

In [3]:
def left_shift(bits, shifts):
    """ Left shift the bits. """
    return bits[shifts:] + bits[:shifts]

In [4]:
def key_schedule(key):
    """ Generate the 16 round keys from the given key. """
    key = permute(key, pc1_table)
    left, right = key[:28], key[28:]
    round_keys = []

    for shift in shift_schedule:
        left = left_shift(left, shift)
        right = left_shift(right, shift)
        round_key = permute(left + right, pc2_table)
        round_keys.append(round_key)

    return round_keys

In [5]:
def s_box_substitution(bits):
    """ Apply S-box substitution. """
    result = []
    for i in range(0, len(bits), 6):
        block = bits[i:i + 6]
        row = (block[0] << 1) + block[5]
        col = (block[1] << 3) + (block[2] << 2) + (block[3] << 1) + block[4]
        s_value = s_boxes[i // 6][row][col]
        result += [int(x) for x in f"{s_value:04b}"]
    return result

In [6]:
def f_function(right, round_key):
    """ The F function used in the DES algorithm. """
    right_expanded = permute(right, e_box_table)
    xor_result = [right_expanded[i] ^ round_key[i] for i in range(48)]
    s_output = s_box_substitution(xor_result)
    return permute(s_output, p_table)

In [7]:
def des_encrypt(plaintext, key):
    """ Encrypt the plaintext using the DES algorithm. """
    round_keys = key_schedule(key)
    plaintext = permute(plaintext, ip_table)
    left, right = plaintext[:32], plaintext[32:]

    for round_key in round_keys:
        temp = right
        right = [left[i] ^ f_function(right, round_key)[i] for i in range(32)]
        left = temp

    combined = left + right
    ciphertext = permute(combined, ip_inv_table)
    return ciphertext

In [8]:
def des_decrypt(ciphertext, key):
    """ Decrypt the ciphertext using the DES algorithm. """
    round_keys = key_schedule(key)[::-1]  # Reverse the order of the keys
    ciphertext = permute(ciphertext, ip_table)
    left, right = ciphertext[:32], ciphertext[32:]

    for round_key in round_keys:
        temp = right
        right = [left[i] ^ f_function(right, round_key)[i] for i in range(32)]
        left = temp

    combined = left + right
    plaintext = permute(combined, ip_inv_table)
    return plaintext

In [9]:
# Example usage
if __name__ == "__main__":
    # Ensure the plaintext and key are exactly 64 bits long
    # Example 64-bit plaintext (8 bytes)
    plaintext = [
        0, 1, 0, 1, 0, 1, 0, 1,  # Byte 1
        0, 1, 0, 1, 0, 1, 0, 1,  # Byte 2
        1, 1, 1, 1, 1, 1, 1, 1,  # Byte 3
        1, 1, 1, 1, 1, 1, 1, 1,  # Byte 4
        0, 0, 0, 0, 0, 0, 0, 0,  # Byte 5
        0, 0, 0, 0, 0, 0, 0, 0,  # Byte 6
        1, 0, 1, 0, 1, 0, 1, 0,  # Byte 7
        1, 0, 1, 0, 1, 0, 1, 0   # Byte 8
    ]
    
    # Example 64-bit key (8 bytes)
    key = [
        1, 0, 1, 0, 1, 0, 1, 0,  # Byte 1
        1, 0, 1, 0, 1, 0, 1, 0,  # Byte 2
        0, 1, 0, 1, 0, 1, 0, 1,  # Byte 3
        0, 1, 0, 1, 0, 1, 0, 1,  # Byte 4
        1, 1, 1, 1, 1, 1, 1, 1,  # Byte 5
        1, 1, 1, 1, 1, 1, 1, 1,  # Byte 6
        0, 0, 0, 0, 0, 0, 0, 0,  # Byte 7
        0, 0, 0, 0, 0, 0, 0, 0   # Byte 8
    ]

    # Check lengths before encryption
    if len(plaintext) != 64:
        raise ValueError("Plaintext must be 64 bits long.")
    if len(key) != 64:
        raise ValueError("Key must be 64 bits long.")

    ciphertext = des_encrypt(plaintext, key)
    print("Ciphertext:", ciphertext)

    decrypted_plaintext = des_decrypt(ciphertext, key)
    print("Decrypted Plaintext:", decrypted_plaintext)

Ciphertext: [0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1]
Decrypted Plaintext: [0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0]


<div style="text-align: center;">
    <h3>Light || DS/22-26/020</h3>
</div>